In [1]:
%pip install langchain-text-splitters langchain-openai langchain-community langchain-core chromadb pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os


In [ ]:
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"  # Substitua pela sua chave da OpenAI



In [4]:
embeddings_model = OpenAIEmbeddings()

llm = ChatOpenAI(model_name="gpt-4o", max_tokens=400)

In [5]:
# Carregar o PDF
pdf_path = "os-sertoes.pdf"
loader = PyPDFLoader(pdf_path, extract_images=False)
pages = loader.load_and_split()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

chunks = text_splitter.split_documents(pages)

In [7]:
db = Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="test_index")
db.persist()

C:\Users\Renan\AppData\Local\Temp\ipykernel_14412\3584155588.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [8]:
# Carregar o vector store existente
vectordb = Chroma(persist_directory="test_index", embedding_function=embeddings_model)

# Configurar o retriever para buscar os 3 documentos mais relevantes
retriever = vectordb.as_retriever(search_kwargs={'k': 3})

C:\Users\Renan\AppData\Local\Temp\ipykernel_14412\1718871071.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory="test_index", embedding_function=embeddings_model)


In [17]:
# Criar o template do prompt
TEMPLATE = """Você é um assistente especializado em literatura brasileira.
Use o seguinte contexto para responder à pergunta de forma detalhada e precisa.

Contexto:
{context}

Pergunta: {question}

Resposta detalhada:"""

prompt = ChatPromptTemplate.from_template(TEMPLATE)




In [10]:
# Função auxiliar para formatar documentos
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# Criar a chain RAG usando LCEL (LangChain Expression Language)
setup_retrieval = RunnableParallel(
    {
        "question": RunnablePassthrough(), 
        "context": retriever | format_docs
    }
)

output_parser = StrOutputParser()

# Montar a chain completa
rag_chain = setup_retrieval | prompt | llm | output_parser



In [11]:
rag_chain.invoke("Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?")

'O contexto fornecido não oferece detalhes específicos sobre a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e sua influência na vida dos habitantes. No entanto, com base em seu famoso livro "Os Sertões", Euclides apresenta o sertão como um ambiente hostil, desafiador e isolado dos centros urbanos do Brasil. Ele descreve a geografia árida e o clima severo como fatores que moldam a vida e a resistência dos habitantes locais, conhecidos como sertanejos. A obra é conhecida por sua análise detalhada da interação entre o meio ambiente inóspito e as pessoas que nele vivem, enfatizando a adaptação e a resiliência dos sertanejos diante das adversidades naturais. Euclides vê o sertão não apenas como um lugar físico, mas como um símbolo das desigualdades sociais e do abandono dos governantes em relação às áreas mais remotas do país.'

In [12]:
rag_chain.invoke("Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?")

'Baseado no contexto fornecido, a resposta não aborda diretamente as características da população sertaneja, mas foca mais em descrever o ambiente e as dificuldades em estudá-lo. Euclides da Cunha menciona que as características geológicas e topográficas, juntamente com outros agentes físicos, interagem de maneira complexa e indefinida, criando um ambiente com influências mútuas e recíprocas contínuas. Ele ressalta a dificuldade em definir qual fator é predominante e menciona a falta de observações científicas suficientes devido à desatenção tradicional em relação ao sertão.\n\nNo geral, Euclides costuma conectar as características da população sertaneja com o ambiente hostil e desafiador em que vivem. Ele frequentemente descreve os sertanejos como resilientes e adaptados às adversidades do sertão, frutos de um contexto onde o ambiente rigoroso exige resistência e adaptação. No entanto, sem trechos específicos sobre a população sertaneja neste contexto, detalhes mais concretos sobre es

In [13]:
rag_chain.invoke("Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?")

'Segundo Euclides da Cunha, o contexto histórico e político que levou à Guerra de Canudos envolveu uma série de fatores sociais, religiosos e políticos complexos. A figura central deste episódio foi Antônio Vicente Mendes Maciel, conhecido como Antônio Conselheiro, que nasceu em Quixeramobim, no Ceará. Ele percorreu o sertão da Bahia construindo igrejas e reformando cemitérios, ações que lhe angariaram seguidores e consolidaram sua influência entre as populações sertanejas.\n\nA comunidade de Belo Monte, formada às margens do Rio Vaza-Barris na região de Canudos, virou o centro das atenções. Para o governo republicano da época, Canudos foi percebida como uma ameaça, visto que foi acusada de representar uma reação monárquica destinada a solapar as instituições republicanas recém-estabelecidas. Essa suspeita de que Canudos era um foco de resistência à República motivou a preparação de expedições punitivas contra a comunidade.\n\nNo livro "Os Sertões", Euclides da Cunha descreve essas exp

In [14]:
rag_chain.invoke("Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?")

'Baseado no contexto fornecido, Euclides da Cunha aborda a figura de Antônio Vicente Mendes Maciel, conhecido como Antônio Conselheiro, como um personagem central e enigmático na narrativa dos eventos relacionados à Guerra de Canudos. Ele é descrito como uma figura carismática e religiosa, nascida em Quixeramobim, Ceará, que dedicou sua vida a percorrer o sertão da Bahia com o intuito de construir igrejas e reformar cemitérios. Essa missão culmina na formação da comunidade de Belo Monte às margens do Rio Vaza-Barris, na região de Canudos.\n\nAntônio Conselheiro é visto como um líder espiritual que atraiu um grande número de seguidores, formando uma comunidade que eventualmente entrou em conflito com o governo republicano do Brasil. As autoridades da época temiam que Canudos representasse uma ameaça monárquica aos ideais republicanos, o que levou ao envio de expedições militares para destruir a comunidade. O papel de Conselheiro, portanto, é central na formação de Canudos, servindo como

In [15]:
rag_chain.invoke("Quais são os principais aspectos da crítica social e política presentes em Os Sertões? Como esses aspectos refletem a visão do autor sobre o Brasil da época?")

'O contexto fornecido não menciona especificamente os aspectos de crítica social e política de "Os Sertões". Contudo, com base no conhecimento geral sobre a obra de Euclides da Cunha, alguns dos principais aspectos da crítica social e política presentes no livro podem ser identificados.\n\n"Os Sertões" é uma obra que aborda a Guerra de Canudos, um conflito entre o exército brasileiro e os habitantes da comunidade de Canudos no interior da Bahia, liderada por Antônio Conselheiro. A partir desse evento, Euclides da Cunha empreende uma análise multifacetada, que entrelaça ciência, geografia, sociologia, história e literatura, apontando profundas críticas sociais e políticas ao Brasil do final do século XIX. \n\n1. **Desigualdade Social e Exclusão**: Euclides da Cunha destaca a condição de miséria e abandono dos sertanejos, vítimas de um Estado que não se fazia presente em termos de assistência e desenvolvimento, mas que era rápido em atuar com violência extrema. A obra denuncia a desigual